In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import cv2
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
import keras
from keras_preprocessing import image
from keras import preprocessing
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.models import Sequential
from keras.layers import Dense , Activation , Dropout ,Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.metrics import categorical_accuracy
from keras.optimizers import *
from keras.layers.normalization import BatchNormalization

In [ ]:
########################################################################################################################
################################################## Parameters ##########################################################
########################################################################################################################
img_width = 224
img_height = 224
img_channel = 3
BATCH_SIZE = 8

########################################################################################################################
################################################ Model building ########################################################
########################################################################################################################
final_model = Sequential()

# 1 - Convolution
final_model.add(Conv2D(64,(3,3), padding='same', input_shape=(48, 48,1)))
final_model.add(BatchNormalization())
final_model.add(Activation('relu'))
final_model.add(MaxPooling2D(pool_size=(2, 2)))
final_model.add(Dropout(0.25))

# 2nd Convolution layer
final_model.add(Conv2D(128,(5,5), padding='same'))
final_model.add(BatchNormalization())
final_model.add(Activation('relu'))
final_model.add(MaxPooling2D(pool_size=(2, 2)))
final_model.add(Dropout(0.25))

# 3rd Convolution layer
final_model.add(Conv2D(512,(3,3), padding='same'))
final_model.add(BatchNormalization())
final_model.add(Activation('relu'))
final_model.add(MaxPooling2D(pool_size=(2, 2)))
final_model.add(Dropout(0.25))


# Flattening
final_model.add(Flatten())

# Fully connected layer 1st layer
final_model.add(Dense(256))
final_model.add(BatchNormalization())
final_model.add(Activation('relu'))
final_model.add(Dropout(0.25))


# Fully connected layer 2nd layer
final_model.add(Dense(512))
final_model.add(BatchNormalization())
final_model.add(Activation('relu'))
final_model.add(Dropout(0.25))

final_model.add(Dense(7, activation='sigmoid'))

#base_model.summary()
final_model.summary()



Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 64)        640       
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 64)        256       
_________________________________________________________________
activation (Activation)      (None, 48, 48, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 128)       204928    
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 128)      

In [ ]:
address = '/home/rd/apply/MMAFEDB/'

train_list = os.listdir(address + 'train')
test_list = os.listdir(address + 'test')

labels_train = []
image_paths_train = []

for i in range(len(train_list)):
  imgs_tr = os.listdir(address + 'train/'+train_list[i])
  for img in imgs_tr:
    image_paths_train.append(address + 'train/'+train_list[i] +'/'+img)
    labels_train.append(i)


labels_test = []
image_paths_test = []

for i in range(len(train_list)):
  imgs_ts = os.listdir(address + 'test/'+train_list[i])
  for img in imgs_ts:
    image_paths_test.append(address + 'test/'+train_list[i] +'/'+img)
    labels_test.append(i)

print(len(labels_test))

training_index = list(np.linspace(0, len(labels_train)-1,len(labels_train), dtype='int'))

print(image_paths_train[:10])
print(labels_train)

import numpy as np
def train_generator(BATCH_SIZE):
    X = np.zeros((BATCH_SIZE, img_width, img_height, img_channel))
    Y = []
    index = 0
    random.shuffle(training_index)
    while(True):
        for i in range(BATCH_SIZE):
            img = cv2.imread(image_paths_train[training_index[index]])
            img = cv2.resize(src=img, dsize=(img_width, img_height))
            img = img/127.5 - 1
            X[i] = img
            Y.append(labels_train[training_index[index]])
            index += 1
            if (index == len(training_index)):
                index = 0
                random.shuffle(training_index)
        yield X, tf.keras.utils.to_categorical(y=Y, num_classes=7)
        X = np.zeros((BATCH_SIZE, img_width, img_height, img_channel))
        Y = []


def test_generator(n):
    X = np.zeros((n, img_width, img_height, img_channel))
    Y = []
    index = np.random.randint(0,17345,n,dtype='int')
    for i in range(n):
        img = cv2.imread(image_paths_test[index[i]])
        img = cv2.resize(src=img, dsize=(img_width, img_height))
        img = img/127.5 - 1
        X[i] = img
        Y.append(labels_test[index[i]])
    return X, tf.keras.utils.to_categorical(y=Y, num_classes=7)
    
        

#x_test, y_test = test_generator()

########################################################################################################################
##################################################### Train ############################################################
########################################################################################################################
monitor = tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=1e-3, patience=50, mode='auto', restore_best_weights=True)
final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
hist = final_model.fit(x=train_generator(BATCH_SIZE), epochs=60, validation_data=test_generator(2300), callbacks=[monitor], steps_per_epoch=11620)
